# LangGraph를 사용한 고객 지원 챗봇을 구축

- 웹 검색을 통해 일반적인 질문에 답변 
- 대화 상태를 유지하여 연속적인 대화  
- 복잡한 질문을 사람이 검토하도록 라우팅  
- 사용자 지정 상태(Custom State)를 활용하여 챗봇의 동작 제어  
- 대화 흐름을 되돌리고(Rewind), 다른 대화 경로 탐색 

In [ ]:
# LangSmith 추적 설정 활성화

먼저 모델을 직접 사용해 봅니다. `ChatModel`은 LangChain의 **"Runnable"** 인스턴스이며, 이는 표준화된 인터페이스를 통해 상호작용할 수 있음을 의미합니다.  

모델을 간단하게 호출하려면 `.invoke` 메서드에 **메시지 목록**을 전달하면 됩니다.

모델 자체는 **상태(state)** 라는 개념을 가지고 있지 않습니다. 예를 들어, 후속 질문을 하면:

<br>
이제 좋은 응답을 받는 것을 확인할 수 있습니다!  

이것이 챗봇이 **대화형 상호작용**을 할 수 있는 기본 아이디어입니다.  

### **Part 1: 기본 챗봇 만들기¶**  

In [ ]:
# MessageState를 기반으로 그래프를 구성하기 위한 graph_builder 설정

다음으로 **"chatbot" 노드**를 추가합니다.  
**노드(Nodes)** 는 작업의 단위를 나타내며, 일반적으로 **일반적인 Python 함수**로 구성됩니다. 

In [ ]:
# 챗봇 함수 정의
def chatbot(state: MessagesState):
# 첫 번째 인자는 노드의 고유한 이름 ("chatbot")
# 두 번째 인자는 노드가 실행될 때 호출될 함수 또는 객체

- 챗봇 노드 함수는 **현재 상태(State)를 입력**으로 받고, **업데이트된 메시지 리스트**를 `"messages"` 키에 담아 **딕셔너리 형태로 반환**합니다.  
- 이 패턴은 **모든 LangGraph 노드 함수의 기본 구조**입니다.  

**다음 단계:**  
- **엔트리 포인트(Entry Point)** 를 추가  
- **엔트리 포인트**는 **그래프 실행 시 시작 지점을 정의**하며, **매번 그래프를 실행할 때 어디서부터 작업을 시작할지 지정하는 역할**을 합니다.
- 마찬가지로, **종료 지점(Finish Point)** 을 설정합니다. 이는 그래프에게 **"이 노드가 실행될 때마다, 여기서 작업을 종료할 수 있다."** 라고 지시하는 역할을 합니다.
- 마지막으로, 그래프를 실행할 수 있도록 설정해야 합니다. 이를 위해 **`compile()`** 메서드를 그래프 빌더에서 호출합니다.   

### **챗봇 실행**  

대화 루프에서 언제든지 **"quit"**, **"exit"**, 또는 **"q"** 를 입력하면 종료할 수 있습니다. 

In [ ]:
def stream_graph_updates(user_input: str):
# 무한 루프 (사용자가 "quit", "exit", "q"를 입력하면 종료)
        # `input()`이 사용 불가능한 경우 예외 처리 (예: 특정 환경에서 실행 시)

LangGraph를 사용하여 첫 번째 챗봇을 구축 했습니다.  

다음 단계에서는 웹 검색 도구를 추가하여 챗봇의 지식을 확장하고, 더 강력한 기능을 갖추도록 만들겠습니다. 

## **Part 2: 도구를 활용한 챗봇 강화**   
이제 챗봇이 **웹에서 관련 정보를 찾아 더 나은 답변을 제공할 수 있도록 개선**합니다. 

---
먼저 **Tavily 검색 엔진**을 사용하기 위해 필요한 패키지를 설치하고 **`TAVILY_API_KEY`** 를 설정 합니다.  

In [ ]:
# %%capture --no-stderr
# %pip install -U tavily-python langchain_community

결과는 **페이지 요약(Summaries)** 으로, **챗봇이 질문에 답변할 때 사용할 수 있습니다.**  

다음 코드는 **Part 1과 동일하지만, 한 가지 차이가 있습니다.**  

**LLM에 `bind_tools`를 추가하여, LLM이 검색 엔진을 사용할 때 올바른 JSON 형식을 인식하도록 설정했습니다.** 

In [ ]:
# 수정 사항: LLM이 사용할 수 있는 도구(Tools)를 명시적으로 설정
# 챗봇 함수 정의
def chatbot(state: MessagesState):

이제 도구가 호출되었을 때 실제로 실행하는 함수를 만들어야 합니다. 이를 위해 새로운 노드(node)를 추가하여 도구를 실행할 수 있도록 설정합니다.

이 노드는 Anthropic, OpenAI, Google Gemini 등 여러 LLM 제공업체에서 지원하는 **`tool_calling` 기능**을 활용합니다.  

In [ ]:
# "tools" 노드 생성 및 그래프에 추가

**조건부 엣지(Conditional Edges)의 동작 방식**  

- 조건부 엣지는 특정 노드에서 시작됩니다. 즉, 'chatbot' 노드가 실행될 때마다, 다음과 같은 경로를 결정합니다.

1️) 만약 챗봇이 도구(`tool_calls`)를 호출하면 'tools' 노드로 이동   
2️) 만약 챗봇이 직접 응답하면 `END`로 이동하여 실행 종료


**`tools_condition`**  

- **`tool_calls`가 없는 경우 `END` 문자열을 반환하여 실행을 종료** 합니다.  

In [ ]:
# 도구 사용이 완료되면 다시 챗봇 노드로 돌아가서 다음 단계 결정
# 그래프의 시작점 정의 (START → chatbot)
# 그래프 컴파일 (최종 그래프 생성)

이제 챗봇에게 **훈련 데이터에 없는 질문도 할 수 있습니다.** 

이제 LangGraph에서 검색 엔진을 활용할 수 있는 대화형 에이전트(Conversational Agent)를 만들었습니다

--- 

## **Part 3: 챗봇에 메모리 기능 추가¶**  

현재 챗봇은 **사용자 질문에 도구를 활용해 답변할 수 있지만, 이전 대화의 맥락을 기억하지 못합니다.**  
이 때문에 **일관된 멀티턴(Multi-turn) 대화를 진행하는 데 한계가 있습니다.**  

LangGraph는 **"지속적 체크포인트(Persistent Checkpointing)"** 기능을 통해 이 문제를 해결합니다.  

그래프를 컴파일할 때 checkpointing을 활성화하고 그래프를 호출할 때 `thread_id`를 제공하면, LangGraph가 자동으로 상태(state)를 저장하고, 다음 실행 시 이전 상태를 복원합니다.  

즉, **동일한 `thread_id`** 를 사용하여 그래프를 호출하면, 이전 대화 상태를 불러와서 이어서 대화할 수 있습니다!

우리는 현재 **메모리를 활용하는(in-memory) 체크포인터**를 사용하고 있습니다.  

이 방식은 튜토리얼 환경에서는 편리하지만, 데이터가 메모리에만 저장되므로 영구적이지 않습니다. 실제 프로덕션 환경에서는 `SqliteSaver` 또는 `PostgresSaver`를 사용하여 데이터베이스(DB)와 연결하는 것이 일반적입니다.

---

## **그래프 정의**

In [ ]:
def chatbot(state: MessagesState):
# # 도구가 호출될 때마다, 다음 단계를 결정하기 위해 챗봇으로 돌아갑니다.
# 제공된 체크포인터와 함께 그래프를 컴파일

이제 챗봇과 상호작용할 수 있습니다. 먼저, 이 대화를 식별할 수 있는 **`thread`(스레드)** 를 선택합니다.

In [ ]:
# config는 stream() 또는 invoke()의 두 번째 인자

이제 후속 질문(follow-up question)을 해봅시다. 챗봇이 사용자의 이름을 기억하는지 확인해 봅니다.

In [ ]:
# `thread_id`를 "1" 대신 "2"로 변경합니다.

---
지금까지 두 개의 서로 다른 thread에서 여러 개의 체크포인트를 생성했습니다. 체크포인트에는 어떤 정보가 저장되는 정보는 다음과 같습니다. 

- 현재 상태 값(Current State Values) 
- 해당 상태와 연결된 `config` 정보  
- 다음으로 처리할 노드(Next Node to Process) 

In [ ]:
# 현재 상태 값

In [ ]:
# 해당 상태와 관련된 config 정보

In [ ]:
# 다음으로 처리할 node

### 프롬프트 템플릿(Prompt Templates)을 사용하여 LLM 호출 최적화

**프롬프트 템플릿** 은 **원시 사용자 입력(raw user input)** 을 LLM이 처리할 수 있는 형식으로 변환하는 데 도움을 줍니다.   

1. 먼저, **시스템 메시지(system message)** 를 추가하여 **사용자 정의 지침(custom instructions)** 을 포함시킵니다. (여전히 메시지를 입력으로 사용)  
2. 다음으로, 메시지 외에 **더 많은 입력 정보** 를 추가합니다.  

#### **시스템 메시지(System Message) 추가하기**

시스템 메시지를 추가하기 위해 **`ChatPromptTemplate`** 을 생성합니다. 여기서는 메시지 전달을 위해 **`MessagesPlaceholder`** 를 사용하겠습니다.  

이렇게 하면 LLM에 전달되는 입력이 더 구조화되고, 챗봇의 동작을 더 정교하게 제어할 수 있습니다.  

In [ ]:
# LLM이 사용자 입력을 더 잘 처리할 수 있도록 프롬프트 템플릿을 설정합니다.
        # LLM의 동작 방식을 정의하는 지침
        # Messages Placeholder - 이전 대화 메시지들을 전달합

이제 이 템플릿을 통합하여 애플리케이션을 업데이트할 수 있습니다.

In [ ]:
# 모델 호출 함수 정의
def chatbot(state: MessagesState):
# START에서 "chatbot" 노드로 이동하도록 엣지 정의
# 그래프를 메모리 체크포인트와 함께 컴파일합니다.

In [ ]:
# 'configurable' 키를 사용하여 추가적인 설정 값을 전달합니다.
# 여기서는 'thread_id'를 사용하여 특정 대화 스레드를 식별합니다.
# 메시지 목록에 사용자 메시지 추가
# 애플리케이션 호출
# 메시지 상태(State), 설정(config) 전달

## **Part 4: 인간 개입**

작업을 성공적으로 수행하려면 **사람의 입력(human input)** 이 필요할 수도 있습니다.  
또한, 특정 작업을 실행하기 전에 **사람의 승인(human approval)** 을 요구하여 올바르게 동작하는지 확인하고 싶을 수도 있습니다.  

---

## **LangGraph의 Human-in-the-loop 지원**  
LangGraph의 **영속성 계층(persistence layer)** 은 **"Human-in-the-loop" 워크플로우** 를 지원합니다.  
이를 통해 **실행을 일시 중지(pause)하고, 사용자 피드백을 기반으로 다시 실행(resume)** 할 수 있습니다.  

In [ ]:
class State(TypedDict):
def human_assistance(query: str) -> str:
def chatbot(state: State):
    # 도구 실행 중에 중단이 발생하므로
    # 재개할 때 도구 호출이 반복되지 않도록 병렬 도구 호출을 비활성화합니다.

이제 새로운 human_assistance 도구를 사용할 수 있는 질문을 챗봇에 입력해 보겠습니다.

--------------------
챗봇이 도구 호출을 생성했지만 실행이 중단되었습니다! 그래프 상태를 검사하면 도구 노드에서 멈췄음을 알 수 있습니다.

Python의 내장 input() 함수와 유사하게, 도구 내부에서 interrupt를 호출하면 실행이 일시 중지됩니다. 

실행을 재개하려면 도구에서 예상하는 데이터가 포함된 Command 객체를 전달합니다. 이 데이터의 형식은 필요에 따라 사용자 정의할 수 있습니다. 여기서는 "data" 키가 있는 dict만 있으면 됩니다.

-----------------
인터럽트를 사용하여 채팅봇에 인간 참여 루프 실행을 추가하여 필요할 때 인간의 감독과 개입을 허용했습니다. 이를 통해 AI 시스템으로 만들 수 있는 잠재적인 UI가 열립니다. 이미 체크포인터를 추가했으므로 기본 지속성 계층이 실행되는 한 그래프를 무기한 일시 중지하고 아무 일도 없었던 것처럼 언제든지 다시 시작할 수 있습니다.